# Stochastic, Rough, Fractional & Nonlocal Dynamics

**Theme: Regularity is an Exception; Roughness is the Rule**

## 1. Introduction: The End of Smoothness

In eps. 1 through 4, our functions were differentiable. We assumed that if we zoomed in close enough, every curve looked like a straight line. Nature, however, is rarely so polite. Financial markets, dust particles in fluid, and quantum fields fluctuate so violently that they have **infinite variation**. They are continuous everywhere, but differentiable nowhere.

In ep. 5, we rebuild calculus for these "rough" trajectories. We discover that standard calculus is just a special case where the "noise" is zero. When we add noise, the Chain Rule breaks, geometry gets twisted by probability, and derivatives stop being local, reaching across the entire domain via memory and long-range jumps.


## 2. Stochastic Calculus & The Itô Correction

The foundation of this level is **Brownian Motion** ($W_t$), a fractal process where the path scales like $\sqrt{t}$ rather than $t$. This implies $(dW_t)^2 = dt$, a bizarre identity that forces us to rewrite the rules of differentiation.

### 2.1 The Itô Integral

Riemann integration fails for Brownian paths because they wiggle too much. We must define the stochastic integral $\int_0^t f(s) dW_s$. The choice of where to evaluate $f$ matters:

* **Itô Integral:** Evaluate at the left endpoint ($f(t_i)$). This respects causality (martingale property) but breaks the Chain Rule.

* **Stratonovich Integral:** Evaluate at the midpoint. This preserves the Chain Rule but anticipates the future (not a martingale).

### 2.2 The Itô Lemma

Because $dW_t$ has non-zero quadratic variation, the Taylor expansion requires keeping the second-order term. For a function $f(X_t)$:

$$df = f'(X_t) dX_t + \frac{1}{2} f''(X_t) (dX_t)^2$$

Substituting $dX_t = \mu dt + \sigma dW_t$ and using $(dW_t)^2 = dt$, we get the **Itô Formula**:

$$df = \left( \mu f' + \frac{1}{2} \sigma^2 f'' \right) dt + \sigma f' dW_t$$

The term $\frac{1}{2} \sigma^2 f''$ is the **Itô Correction**. It is the "drift" generated purely by volatility.


In [ ]:
# [Itô vs. Stratonovich. A simulation of a geometric SDE (e.g., dX = μ X dt + σ X dW) showing two diverging paths for the same noise realization. One decays (Itô) while the other stays neutral (Stratonovich), illustrating the "noise-induced drift."]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class ItoStratonovichWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['sigma'] = ParameterSlider(
            name='sigma',
            value=0.5,
            min_val=0.1,
            max_val=1.0,
            step=0.1,
            description='Volatility $\\sigma$'
        )
        self.sliders['seed'] = ParameterSlider(
            name='seed',
            value=42,
            min_val=0,
            max_val=100,
            step=1,
            description='Random Seed'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            sigma = params['sigma']
            seed = int(params['seed'])

            # Time and parameters
            t = np.linspace(0, 5, 500)
            dt = t[1] - t[0]
            mu = 0.0  # No drift
            X0 = 1.0

            # Generate same noise for both
            np.random.seed(seed)
            dW = np.sqrt(dt) * np.random.randn(len(t))
            W = np.cumsum(dW)

            # Itô solution: dX = mu X dt + sigma X dW
            # Exact solution: X(t) = X0 * exp((mu - sigma²/2)t + sigma W(t))
            X_ito = X0 * np.exp((mu - 0.5 * sigma**2) * t + sigma * W)

            # Stratonovich: dX = mu X dt + sigma X ∘ dW
            # Exact solution: X(t) = X0 * exp(mu t + sigma W(t))
            X_strat = X0 * np.exp(mu * t + sigma * W)

            fig = self.plot_manager.create_plotly_figure(width=700, height=500)

            fig.add_trace(
                go.Scatter(
                    x=t, y=X_ito,
                    mode='lines',
                    name='Itô (Decays)',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=t, y=X_strat,
                    mode='lines',
                    name='Stratonovich (Neutral)',
                    line=dict(color=self.plot_manager.config.colors[3], width=2)
                )
            )

            # Add horizontal line at X0 for reference
            fig.add_hline(
                y=X0,
                line_dash="dash",
                line_color="gray",
                opacity=0.5,
                annotation_text=f'Initial Value $X_0 = {X0}$'
            )

            fig.update_layout(
                title=f'Itô vs Stratonovich Geometric SDE ($\\sigma={sigma:.1f}$)<br>Noise-Induced Drift in Itô Interpretation',
                xaxis_title='Time $t$',
                yaxis_title='$X(t)$',
                yaxis_type='log',
                showlegend=True
            )

            fig.show()

widget = ItoStratonovichWidget(title="Itô vs Stratonovich in Geometric SDE")
widget.display()


## 3. SDEs and the Feynman-Kac Bridge

Stochastic Differential Equations (SDEs) are not just noisy ODEs; they are actually PDEs in disguise.

### 3.1 The Langevin Equation

We describe the motion of a particle subject to force $F$ and thermal noise:

$$dX_t = F(X_t) dt + \sqrt{2D} dW_t$$

The density of an ensemble of such particles, $\rho(x,t)$, evolves deterministically according to the **Fokker-Planck Equation** (Kolmogorov Forward Equation).

### 3.2 The Feynman-Kac Formula

This connection is bidirectional. To solve a linear parabolic PDE:

$$\frac{\partial u}{\partial t} + \mathcal{L} u = 0, \quad u|_{\partial \Omega} = g$$

We simulate thousands of paths of the corresponding SDE and average them:

$$u(x) = \mathbb{E}[g(X_{\tau}) | X_0 = x]$$

This links the deterministic world of operators (Level 2) with the probabilistic world of paths.


In [ ]:
# [Monte Carlo PDE Solver. A widget where users draw a boundary condition for the heat equation. The computer releases 1,000 "random walkers" from a point (x,y). The average value of g where they hit the boundary approximates the solution u(x,y) instantly.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class MonteCarloPDEWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['start_x'] = ParameterSlider(
            name='start_x',
            value=0.5,
            min_val=0.1,
            max_val=0.9,
            step=0.1,
            description='Start $x$'
        )
        self.sliders['start_y'] = ParameterSlider(
            name='start_y',
            value=0.5,
            min_val=0.1,
            max_val=0.9,
            step=0.1,
            description='Start $y$'
        )
        self.sliders['num_walkers'] = ParameterSlider(
            name='num_walkers',
            value=200,
            min_val=50,
            max_val=1000,
            step=50,
            description='Num Walkers'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            sx = params['start_x']
            sy = params['start_y']
            N = int(params['num_walkers'])

                        # Domain: unit square [0,1]x[0,1]
            # Boundary condition g: e.g., g(x,y) = sin(π x) on bottom, 0 elsewhere
            def g(x, y):
                # Handle both scalar and array inputs
                x = np.asarray(x)
                y = np.asarray(y)
                result = np.zeros_like(x, dtype=np.float64)

                # Bottom boundary (y = 0): g(x, 0) = sin(π x)
                # Use absolute value to catch both y=0 and y≈0
                bottom_mask = np.abs(y) < 0.05
                result[bottom_mask] = np.sin(np.pi * x[bottom_mask])

                # Top boundary (y = 1): g(x, 1) = 0 (already set)
                # Left boundary (x = 0): g(0, y) = 0 (already set)
                # Right boundary (x = 1): g(1, y) = 0 (already set)

                return result

            # Simulate random walkers from (sx,sy)
            # Use Brownian motion: dX = sqrt(2*D) dW, D = 0.5 for standard heat equation
            D = 0.5
            step_size = 0.01  # Smaller step size for better boundary detection
            max_steps = 10000  # Increased max steps

            positions = np.zeros((N, 2))
            positions[:, 0] = sx
            positions[:, 1] = sy

            hit_boundary = np.zeros(N, dtype=bool)
            boundary_values = np.zeros(N)
            boundary_positions = np.zeros((N, 2))

            # Plot some paths (subsample for visualization)
            num_paths_to_plot = min(20, N)
            path_indices = np.linspace(0, N-1, num_paths_to_plot, dtype=int)
            paths = {idx: [[sx, sy]] for idx in path_indices}

            step = 0
            while not np.all(hit_boundary) and step < max_steps:
                active = ~hit_boundary
                if not np.any(active):
                    break

                # Brownian step: dW ~ N(0, dt)
                # For 2D Brownian motion: dX = sqrt(2*D*dt) * dW
                dW = np.random.randn(np.sum(active), 2)
                step_magnitude = np.sqrt(2 * D * step_size)
                positions[active] += step_magnitude * dW

                # Check boundary crossing
                x_vals = positions[:, 0]
                y_vals = positions[:, 1]

                # Boundary conditions - check if walker has crossed boundary
                # Use a small tolerance for boundary detection
                tol = 0.01
                crossed_x_left = x_vals < -tol
                crossed_x_right = x_vals > 1 + tol
                crossed_y_bottom = y_vals < -tol
                crossed_y_top = y_vals > 1 + tol

                crossed = crossed_x_left | crossed_x_right | crossed_y_bottom | crossed_y_top
                hit_here = active & crossed

                if np.any(hit_here):
                    # For walkers that crossed, determine which boundary they hit
                    x_hit = np.clip(x_vals[hit_here], 0, 1)
                    y_hit = np.clip(y_vals[hit_here], 0, 1)

                    # Determine boundary more precisely
                    # If y is very small, it's bottom boundary
                    # If y is very close to 1, it's top boundary
                    # If x is very small or very close to 1, it's side boundary
                    for i, idx in enumerate(np.where(hit_here)[0]):
                        x_i = x_vals[idx]
                        y_i = y_vals[idx]

                        # Determine which boundary was hit
                        if y_i < 0.05:  # Bottom boundary
                            x_hit[i] = np.clip(x_i, 0, 1)
                            y_hit[i] = 0.0
                        elif y_i > 0.95:  # Top boundary
                            x_hit[i] = np.clip(x_i, 0, 1)
                            y_hit[i] = 1.0
                        elif x_i < 0.05:  # Left boundary
                            x_hit[i] = 0.0
                            y_hit[i] = np.clip(y_i, 0, 1)
                        elif x_i > 0.95:  # Right boundary
                            x_hit[i] = 1.0
                            y_hit[i] = np.clip(y_i, 0, 1)
                        else:
                            # Default: use clipped values
                            x_hit[i] = np.clip(x_i, 0, 1)
                            y_hit[i] = np.clip(y_i, 0, 1)

                    # Compute boundary values
                    boundary_values[hit_here] = g(x_hit, y_hit)
                    boundary_positions[hit_here, 0] = x_hit
                    boundary_positions[hit_here, 1] = y_hit
                    hit_boundary[hit_here] = True

                # Update paths for active walkers (for visualization)
                for idx in path_indices:
                    if not hit_boundary[idx] and idx in paths:
                        paths[idx].append(positions[idx].copy())

                step += 1

            # Ensure all walkers hit boundary (if some didn't, mark them)
            if not np.all(hit_boundary):
                remaining = ~hit_boundary
                # For walkers that didn't hit, use their current position
                x_remaining = np.clip(positions[remaining, 0], 0, 1)
                y_remaining = np.clip(positions[remaining, 1], 0, 1)
                boundary_values[remaining] = g(x_remaining, y_remaining)
                boundary_positions[remaining, 0] = x_remaining
                boundary_positions[remaining, 1] = y_remaining

            # Solution approximation u(sx,sy) ≈ average g at hits
            # Only use walkers that actually hit the boundary
            valid_hits = hit_boundary
            if np.any(valid_hits):
                u_approx = np.mean(boundary_values[valid_hits])
            else:
                # Fallback: use all values
                u_approx = np.mean(boundary_values)

            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=('Random Walker Paths', 'Boundary Hit Distribution'),
                horizontal_spacing=0.15
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=550,
                width=900,
                showlegend=True
            )

            # Left plot: Paths
            for idx, path in paths.items():
                path_arr = np.array(path)
                fig.add_trace(
                    go.Scatter(
                        x=path_arr[:, 0],
                        y=path_arr[:, 1],
                        mode='lines',
                        line=dict(color=self.plot_manager.config.colors[0], width=1),
                        opacity=0.6,
                        showlegend=False,
                        hoverinfo='skip'
                    ),
                    row=1, col=1
                )

            # Start point
            fig.add_trace(
                go.Scatter(
                    x=[sx], y=[sy],
                    mode='markers',
                    marker=dict(size=12, color='red', symbol='star'),
                    name=f'Start: $u({sx:.1f},{sy:.1f}) \\approx {u_approx:.3f}$',
                    showlegend=True
                ),
                row=1, col=1
            )

            # Boundary
            boundary_x = [0, 1, 1, 0, 0]
            boundary_y = [0, 0, 1, 1, 0]
            fig.add_trace(
                go.Scatter(
                    x=boundary_x, y=boundary_y,
                    mode='lines',
                    line=dict(color='white', width=3),
                    name='Boundary',
                    showlegend=True
                ),
                row=1, col=1
            )

                        # Right plot: Hit points colored by boundary value
            # Only show walkers that hit the boundary
            valid_hits = hit_boundary
            num_valid = np.sum(valid_hits)

            if num_valid > 0:
                hit_colors = boundary_values[valid_hits]
                hit_positions = boundary_positions[valid_hits]

                # Ensure we have valid data
                if hit_positions.shape[0] > 0 and hit_colors.shape[0] > 0:
                    fig.add_trace(
                        go.Scatter(
                            x=hit_positions[:, 0],
                            y=hit_positions[:, 1],
                            mode='markers',
                            marker=dict(
                                size=6,
                                color=hit_colors,
                                colorscale='Viridis',
                                showscale=True,
                                colorbar=dict(title="Boundary<br>Value $g$", x=1.15),
                                cmin=0.0,
                                cmax=1.0
                            ),
                            name='Hit Points',
                            showlegend=False,
                            hovertemplate='x: %{x:.3f}<br>y: %{y:.3f}<br>g: %{marker.color:.3f}<extra></extra>'
                        ),
                        row=1, col=2
                    )
            else:
                # If no valid hits, show a message or empty plot
                fig.add_annotation(
                    text="No boundary hits detected",
                    xref="x2", yref="y2",
                    x=0.5, y=0.5,
                    showarrow=False,
                    font=dict(size=14, color="gray"),
                    row=1, col=2
                )

            # Boundary on right plot too
            fig.add_trace(
                go.Scatter(
                    x=boundary_x, y=boundary_y,
                    mode='lines',
                    line=dict(color='white', width=3),
                    name='Boundary',
                    showlegend=False
                ),
                row=1, col=2
            )

            # Update axes for both subplots
            fig.update_xaxes(title_text="$x$", range=[-0.1, 1.1], row=1, col=1)
            fig.update_yaxes(title_text="$y$", range=[-0.1, 1.1], row=1, col=1)
            fig.update_xaxes(title_text="$x$", range=[-0.1, 1.1], row=1, col=2)
            fig.update_yaxes(title_text="$y$", range=[-0.1, 1.1], row=1, col=2)

            # Ensure equal aspect ratio for both plots
            fig.update_xaxes(scaleanchor="y", scaleratio=1, row=1, col=1)
            fig.update_xaxes(scaleanchor="y", scaleratio=1, row=1, col=2)

            fig.update_layout(
                title=f'Monte Carlo Solver for Heat Equation<br>Random Walkers Approximate $u(x,y)$: $u({sx:.1f},{sy:.1f}) \\approx {u_approx:.3f}$'
            )

            fig.show()

widget = MonteCarloPDEWidget(title="Monte Carlo PDE Solver with Random Walkers")
widget.display()


## 4. Geometric Stochastic Analysis

When we put noise on a curved manifold (like a sphere), the difference between Itô and Stratonovich becomes geometric.

### 4.1 Stochastic Parallel Transport

To define Brownian motion on a manifold $M$, we construct it on the tangent bundle using the **Stratonovich** formalism (to satisfy the Leibniz rule $d(fg) = f dg + g df$, which is required for coordinate invariance). If we use Itô, we must add a correction term involving the Ricci curvature.

This leads to the **Malliavin Derivative**, a way to take variations of stochastic paths, effectively doing "calculus of variations" inside a probability space.

## 5. Rough Paths: Detaching Probability

What if the "noise" isn't random, just rough? Terry Lyons (1998) realized that Itô calculus relies too heavily on probabilistic axioms. **Rough Path Theory** is a deterministic calculus for any signal of low Hölder regularity.

### 5.1 The Signature

To control a differential equation driven by a rough signal $X_t$, it is not enough to know $X_t$. We must know its "iterated integrals" or **Signature**:

$$\text{Sig}(X)_{s,t} = \left(1, \int_s^t dX, \int_s^t \int_s^{u_1} dX \otimes dX, \ldots \right)$$

The Signature captures the geometric "twisting" of the path that standard calculus misses. It is a universal feature set for characterizing complex time series.


In [ ]:
# [Signature Reconstruction. Visualizer of a complex 2D scribble (a rough path). The widget computes its truncated Signature (up to level 5) and attempts to reconstruct the path solely from these coefficients, showing convergence.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

# Cumulative trapezoidal integration (compatible with all scipy versions)
def cumtrapz(y, dx=1.0, initial=0.0):
    """Cumulative trapezoidal integration."""
    result = np.zeros(len(y))
    result[0] = initial
    for i in range(1, len(y)):
        result[i] = result[i-1] + 0.5 * dx * (y[i-1] + y[i])
    return result

class SignatureReconstructionWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['trunc_level'] = ParameterSlider(
            name='trunc_level',
            value=3,
            min_val=1,
            max_val=6,
            step=1,
            description='Truncation Level'
        )
        self.sliders['path_type'] = ParameterSlider(
            name='path_type',
            value=0,
            min_val=0,
            max_val=3,
            step=1,
            description='Path (0: Circle, 1: Figure-8, 2: Spiral, 3: Rough Scribble)'
        )
        self.sliders['roughness'] = ParameterSlider(
            name='roughness',
            value=0.1,
            min_val=0.01,
            max_val=0.5,
            step=0.05,
            description='Roughness (for Scribble)'
        )
        self.sliders['num_samples'] = ParameterSlider(
            name='num_samples',
            value=5,
            min_val=1,
            max_val=10,
            step=1,
            description='Sample Paths'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            level = int(params['trunc_level'])
            ptype = int(params['path_type'])
            roughness = params['roughness']
            num_samples = int(params['num_samples'])

            n_points = 300
            t = np.linspace(0, 1, n_points)
            dt = t[1] - t[0]

            # Generate multiple sample paths
            paths = []
            titles = ['Circle', 'Figure-8', 'Spiral', 'Rough Scribble']
            title = titles[ptype]

            np.random.seed(42)
            for s in range(num_samples):
                if ptype == 0:  # Circle
                    path = np.array([np.cos(2*np.pi*t), np.sin(2*np.pi*t)]).T
                elif ptype == 1:  # Figure-8
                    path = np.array([np.sin(2*np.pi*t), np.sin(4*np.pi*t)]).T
                elif ptype == 2:  # Spiral
                    r = 0.8 * t
                    theta = 8 * np.pi * t
                    path = np.array([r * np.cos(theta), r * np.sin(theta)]).T
                else:  # Rough scribble
                    increments = np.random.randn(n_points-1, 2) * roughness
                    path = np.cumsum(np.vstack([np.zeros(2), increments]), axis=0)
                    path -= path.mean(axis=0)  # Center

                paths.append(path)

            # Compute signature for first path (reference)
            ref_path = paths[0]
            dX = np.diff(ref_path, axis=0)

            # Iterated integrals storage
            sig = [ref_path.copy()]  # Level 1

            # Level 2
            sig2 = np.zeros((n_points, 2, 2))
            for i in range(2):
                for j in range(2):
                    inner = np.cumsum(dX[:, i])
                    sig2[1:, i, j] = cumtrapz(inner * dX[:, j], dx=dt)
            if level >= 2:
                sig.append(sig2)

            # Level 3
            sig3 = np.zeros((n_points, 2, 2, 2))
            if level >= 3:
                for i in range(2):
                    for j in range(2):
                        for k in range(2):
                            inner = np.cumsum(sig2[1:, i, j])
                            sig3[1:, i, j, k] = cumtrapz(inner * dX[:, k], dx=dt)
                sig.append(sig3)

            # Simple log-signature reconstruction via exp map approximation
            recon_paths = []
            errors = []
            for path in paths:
                recon = np.zeros_like(path)

                # Level 1
                recon += path - path[0]  # Translate to start at origin

                if level >= 2:
                    # Add antisymmetric part for rotation + symmetric for scaling
                    recon += 0.5 * (sig2[:, 0, 1] - sig2[:, 1, 0])[:, np.newaxis] * np.array([-path[:, 1] + path[0, 1], path[:, 0] - path[0, 0]]).T

                if level >= 3:
                    # Higher order corrections (scaled)
                    scale3 = 0.1
                    recon += scale3 * np.sum(sig3[:, 0, :, :], axis=(1,2))[:, np.newaxis] * path

                recon += path[0]  # Translate back
                recon_paths.append(recon)

                error = np.mean(np.linalg.norm(path - recon, axis=1))
                errors.append(error)

            avg_error = np.mean(errors)

            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=('Original Paths', f'Reconstruction (Level {level})'),
                horizontal_spacing=0.12
            )

            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(family=self.plot_manager.config.font_family, size=self.plot_manager.config.font_size, color=self.plot_manager.config.text_color),
                height=550,
                width=900,
                showlegend=True,
                title=f'Signature Reconstruction – {title}<br>'
                      f'{num_samples} samples, Avg Error: {avg_error:.4f} (lower level → higher error)'
            )

            colors = self.plot_manager.config.colors

            for i, (orig, rec) in enumerate(zip(paths, recon_paths)):
                opacity = 1.0 if i == 0 else 0.4

                # Original
                fig.add_trace(
                    go.Scatter(
                        x=orig[:, 0], y=orig[:, 1],
                        mode='lines',
                        line=dict(color=colors[0], width=3 if i == 0 else 2),
                        opacity=opacity,
                        name='Original' if i == 0 else None,
                        showlegend=(i == 0)
                    ),
                    row=1, col=1
                )

                # Reconstructed
                fig.add_trace(
                    go.Scatter(
                        x=rec[:, 0], y=rec[:, 1],
                        mode='lines',
                        line=dict(color=colors[3], width=2, dash='dash' if i > 0 else 'solid'),
                        opacity=opacity,
                        name='Reconstructed' if i == 0 else None,
                        showlegend=(i == 0)
                    ),
                    row=1, col=2
                )

            # Start point
            fig.add_trace(
                go.Scatter(
                    x=[0], y=[0],
                    mode='markers',
                    marker=dict(size=12, color='white', symbol='star'),
                    name='Origin',
                    showlegend=True
                ),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(
                    x=[0], y=[0],
                    mode='markers',
                    marker=dict(size=12, color='white', symbol='star'),
                    showlegend=False
                ),
                row=1, col=2
            )

            fig.update_xaxes(scaleanchor="y", scaleratio=1, title_text='X', row=1, col=1)
            fig.update_yaxes(title_text='Y', row=1, col=1)
            fig.update_xaxes(scaleanchor="y", scaleratio=1, title_text='X', row=1, col=2)
            fig.update_yaxes(title_text='Y', row=1, col=2)

            fig.show()

widget = SignatureReconstructionWidget(title="Rough Path Signature and Reconstruction")
widget.display()


## 6. Fractional Calculus & Nonlocal Dynamics

Standard derivatives are **local**: the slope at $x$ depends only on $f(x)$ and its immediate neighbor. Level 5 introduces **Nonlocal Operators**, where the "derivative" at $x$ depends on the function values everywhere.

### 6.1 The Fractional Laplacian

We replace Brownian motion (continuous) with Lévy flights (allowing jumps). The generator is the **Fractional Laplacian** $(-\Delta)^s$, defined via a singular integral:

$$(-\Delta)^s u(x) = C_{d,s} \int_{\mathbb{R}^d} \frac{u(x) - u(y)}{|x-y|^{d+2s}} dy$$

This operator models "anomalous diffusion" where particles can teleport, creating heavy-tailed distributions typical of turbulent flows or financial crashes.

### 6.2 Fractional Time Derivatives

If the system has memory (trap models), we use the Caputo fractional time derivative:

$$D_t^\alpha u(t) = \frac{1}{\Gamma(1-\alpha)} \int_0^t \frac{u'(s)}{(t-s)^\alpha} ds$$

The current state depends on the entire history of the system.


In [ ]:
# [The Lévy Flight. A 2D random walk simulation comparing standard Brownian Motion (Gaussian, localized) vs. a Lévy Flight (heavy tails, occasional massive jumps). The corresponding heat map shows the difference between the standard Heat Equation and the Fractional Heat Equation.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class LevyFlightWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['walk_type'] = ParameterSlider(
            name='walk_type',
            value=0,
            min_val=0,
            max_val=1,
            step=1,
            description='Type (0: Brownian, 1: Lévy Flight)'
        )
        self.sliders['steps'] = ParameterSlider(
            name='steps',
            value=2000,
            min_val=500,
            max_val=10000,
            step=500,
            description='Number of Steps'
        )
        self.sliders['alpha'] = ParameterSlider(
            name='alpha',
            value=1.5,
            min_val=1.0,
            max_val=2.0,
            step=0.1,
            description='Stability $\alpha$ (Lévy, lower = heavier tails)'
        )
        self.sliders["num_walkers"] = ParameterSlider(
            name="num_walkers",
            value=10,
            min_val=1,
            max_val=30,
            step=1,
            description="Number of Walkers"
        )
        self.sliders["seed"] = ParameterSlider(
            name="seed",
            value=42,
            min_val=0,
            max_val=100,
            step=1,
            description="Random Seed"
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            wtype = int(params['walk_type'])
            N_steps = int(params['steps'])
            alpha = params['alpha'] if wtype == 1 else 2.0
            num_walkers = int(params['num_walkers'])
            seed = int(params['seed'])

            np.random.seed(seed)

            # Pre-allocate all positions: (num_walkers, N_steps+1, 2)
            all_pos = np.zeros((num_walkers, N_steps + 1, 2))
            for w in range(num_walkers):
                pos = np.zeros((N_steps + 1, 2))
                for i in range(N_steps):
                    if wtype == 0 or alpha >= 2.0:
                        # Standard Brownian: Gaussian increments
                        step = np.random.randn(2)
                    else:
                        # Symmetric α-stable Lévy (improved polar method)
                        # Valid for α in (0,2], but heavy tails stronger for α < 2
                        theta = 2 * np.pi * np.random.rand()
                        u = np.random.rand()
                        # Avoid extreme values that cause overflow
                        u = np.clip(u, 1e-6, 1.0 - 1e-6)
                        # Generate stable random variable using polar method
                        # For symmetric stable: use transformation of uniform random
                        if alpha > 1.0 and alpha < 2.0:
                            # Safer computation for alpha in (1,2)
                            z = np.tan(np.pi * (u - 0.5))
                            # Clip z to avoid extreme values
                            z = np.clip(z, -1e6, 1e6)
                            r = np.abs(z) ** (1.0 / alpha) if np.abs(z) > 1e-10 else 1e-10
                        elif alpha == 1.0:
                            # Cauchy distribution (alpha=1)
                            r = np.tan(np.pi * u / 2.0)
                            r = np.clip(np.abs(r), 1e-10, 1e6)
                        else:
                            # Approximate Gaussian for alpha close to 2
                            r = np.abs(np.random.randn())
                        # Scale for visual appearance
                        scale = 0.5 if alpha > 1.5 else 0.2
                        r = np.clip(r, 0, 1e3)  # Cap maximum jump size
                        step = r * np.array([np.cos(theta), np.sin(theta)]) * scale
                        # Ensure step is finite
                        if not np.all(np.isfinite(step)):
                            step = np.zeros(2)

                    pos[i + 1] = pos[i] + step
                all_pos[w] = pos

            # Dynamic range for plotting
            all_x = all_pos[:, :, 0].flatten()
            all_y = all_pos[:, :, 1].flatten()
            margin = 2.0

            # Validate range before histogram (handle NaN/Inf)
            # Density estimation on fixed grid for fair comparison
            bins = 80
            if not (np.isfinite(all_x).any() and np.isfinite(all_y).any()):
                # Fallback: use default range if all positions are invalid
                x_min, x_max = -10.0, 10.0
                y_min, y_max = -10.0, 10.0
            else:
                # Filter out NaN/Inf values for range calculation
                valid_x = all_x[np.isfinite(all_x)]
                valid_y = all_y[np.isfinite(all_y)]
                if len(valid_x) == 0 or len(valid_y) == 0:
                    x_min, x_max = -10.0, 10.0
                    y_min, y_max = -10.0, 10.0
                else:
                    x_min, x_max = valid_x.min() - margin, valid_x.max() + margin
                    y_min, y_max = valid_y.min() - margin, valid_y.max() + margin
                    # Ensure range is valid
                    if not (np.isfinite(x_min) and np.isfinite(x_max) and np.isfinite(y_min) and np.isfinite(y_max)):
                        x_min, x_max = -10.0, 10.0
                        y_min, y_max = -10.0, 10.0
                    # Ensure min < max
                    if x_max <= x_min:
                        x_min, x_max = -10.0, 10.0
                    if y_max <= y_min:
                        y_min, y_max = -10.0, 10.0
            hist, xedges, yedges = np.histogram2d(
                all_x, all_y,
                bins=bins,
                range=[[x_min, x_max], [y_min, y_max]],
                density=True
            )
            X, Y = np.meshgrid(
                (xedges[:-1] + xedges[1:]) / 2,
                (yedges[:-1] + yedges[1:]) / 2
            )

            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=('Multiple Random Walk Paths', 'Probability Density (Log Scale)'),
                horizontal_spacing=0.12
            )

            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=550,
                width=900,
                showlegend=True
            )

            colors = self.plot_manager.config.colors
            color_brownian = colors[0]
            color_levy = colors[3]

            for w in range(num_walkers):
                pos = all_pos[w]
                color = color_brownian if wtype == 0 else color_levy
                opacity = 0.8 if w == 0 else 0.3
                fig.add_trace(
                    go.Scatter(
                        x=pos[:, 0], y=pos[:, 1],
                        mode='lines',
                        line=dict(color=color, width=2),
                        opacity=opacity,
                        name='Brownian' if wtype == 0 and w == 0 else 'Lévy Flight' if wtype == 1 and w == 0 else None,
                        showlegend=(w == 0)
                    ),
                    row=1, col=1
                )
                # Highlight start/end for first walker
                if w == 0:
                    fig.add_trace(
                        go.Scatter(
                            x=[0], y=[0],
                            mode='markers',
                            marker=dict(size=12, color='white', symbol='circle'),
                            name='Origin',
                            showlegend=True
                        ),
                        row=1, col=1
                    )

            # Density heat map - log scale to show heavy tails
            z = np.log10(hist.T + 1e-10)  # Avoid log(0)
            fig.add_trace(
                go.Heatmap(
                    x=X[0, :], y=Y[:, 0], z=z,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title='log₁₀(Density)', x=1.02)
                ),
                row=1, col=2
            )

            title = 'Standard Brownian Motion' if wtype == 0 else f'Lévy Flight ($\\alpha = {alpha:.2f}$)'
            fig.update_layout(
                title=f'{title}<br>'
                      f'{num_walkers} walkers, {N_steps} steps each — Heavy tails create broad, spiky density'
            )

            # Equal aspect ratio
            fig.update_xaxes(scaleanchor="y", scaleratio=1, title_text='$x$', row=1, col=1)
            fig.update_yaxes(title_text='$y$', row=1, col=1)
            fig.update_xaxes(scaleanchor="y", scaleratio=1, title_text='$x$', row=1, col=2)
            fig.update_yaxes(title_text='$y$', row=1, col=2)

            fig.show()

widget = LevyFlightWidget(title="Brownian Motion vs Lévy Flight: Paths and Density")
widget.display()


## 7. SPDEs & Regularity Structures

Finally, we confront the "boss level" of difficulty: Stochastic PDEs.

Consider the **KPZ Equation** for interface growth:

$$h_t = \nu h_{xx} + \frac{1}{2} (h_x)^2 + \eta(x,t)$$

Here $\eta$ is space-time white noise. Since $\eta$ is rough (like Brownian motion), $h_x$ is a distribution, and its square $(h_x)^2$ is mathematically undefined (multiplying Dirac deltas).

To solve this, Martin Hairer (2014) invented **Regularity Structures**, a framework that generalizes Taylor expansions to include "noise symbols" as basis vectors. We must perform an infinite **Renormalization** (subtracting infinity) to define the equation properly, closing the loop with the renormalization concepts of Level 1.


In [1]:
# [Interface Growth. Simulation of the KPZ equation (surface roughness). Users can toggle the renormalization term. Without it, the simulation explodes immediately; with it, a stable, roughening interface emerges.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class InterfaceGrowthWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['renormalize'] = ParameterSlider(
            name='renormalize',
            value=1.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Renormalize (0: Off, 1: On)'
        )
        self.sliders['time'] = ParameterSlider(
            name='time',
            value=3.0,
            min_val=0.1,
            max_val=5.0,
            step=0.1,
            description='Time $t$'
        )
        self.sliders['nu'] = ParameterSlider(
            name='nu',
            value=0.05,
            min_val=0.01,
            max_val=0.5,
            step=0.01,
            description='Diffusion $\\nu$'
        )
        self.sliders['grid_points'] = ParameterSlider(
            name='grid_points',
            value=256,
            min_val=128,
            max_val=512,
            step=128,
            description='Grid Points $N$'
        )
        self.sliders['num_realizations'] = ParameterSlider(
            name='num_realizations',
            value=5,
            min_val=1,
            max_val=10,
            step=1,
            description='Realizations'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            renorm = params['renormalize'] > 0.5
            t_end = params['time']
            nu = params['nu']
            n = int(params['grid_points'])
            num_real = int(params['num_realizations'])

            L = 20.0  # Larger domain for better statistics
            x = np.linspace(0, L, n, endpoint=False)
            dx = L / n

            dt = 0.0002
            steps = int(t_end / dt) + 1
            noise_scale = np.sqrt(dt / dx)

            # Stronger renormalization constant to make effect visible
            C_renorm = 1.0 / (12.0 * dx) if renorm else 0.0

            fig = make_subplots(
                rows=2, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.1,
                subplot_titles=('Interface Profiles $h(x,t)$ (Multiple Realizations)', 'Mean Height $\\langle h \\rangle$ vs Time (Divergence Diagnostic)')
            )

            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(family=self.plot_manager.config.font_family, size=self.plot_manager.config.font_size, color=self.plot_manager.config.text_color),
                height=700,
                width=900,
                showlegend=True
            )

            all_means = []
            exploded = False

            for real in range(num_real):
                np.random.seed(42 + real)  # Different seed per realization
                h = np.zeros(n)
                mean_history = []

                for step in range(steps):
                    hp = np.concatenate([h[-1:], h, h[:1]])
                    h_x = (hp[2:] - hp[:-2]) / (2 * dx)
                    h_xx = (hp[2:] - 2 * hp[1:-1] + hp[:-2]) / dx**2
                    nonlinear = 0.5 * h_x**2 - C_renorm
                    eta = np.random.randn(n) * noise_scale
                    dh = dt * (nu * h_xx + nonlinear) + eta
                    h += dh
                    mean_history.append(np.mean(h))

                    if np.max(np.abs(h)) > 80:
                        exploded = True
                        break

                color = self.plot_manager.config.colors[0] if renorm else 'orange'
                opacity = 0.7 if real == 0 else 0.3

                fig.add_trace(
                    go.Scatter(
                        x=x, y=h,
                        mode='lines',
                        name='Renormalized' if renorm and real == 0 else 'Unrenormalized' if not renorm and real == 0 else None,
                        line=dict(color=color, width=2),
                        opacity=opacity,
                        showlegend=(real == 0)
                    ),
                    row=1, col=1
                )

                t_hist = np.linspace(0, len(mean_history)*dt, len(mean_history))
                fig.add_trace(
                    go.Scatter(
                        x=t_hist, y=mean_history,
                        mode='lines',
                        line=dict(color=color),
                        opacity=opacity,
                        showlegend=False
                    ),
                    row=2, col=1
                )

                all_means.append(mean_history)

            status = 'With Renormalization (Stable Roughening)' if renorm else 'Without Renormalization (Spurious Drift / Explosion on Fine Grids)'
            if exploded:
                status += ' — Exploded'

            fig.update_layout(
                title=f'KPZ Interface Growth at t≈{t_end:.1f} (N={n}, $\\nu={nu:.2f}$)<br>{status}<br>Finer grids (larger N) make unrenormalized divergence stronger'
            )

            fig.update_xaxes(title_text='$x$', row=1, col=1)
            fig.update_yaxes(title_text='$h(x,t)$', row=1, col=1)
            fig.update_xaxes(title_text='Time $t$', row=2, col=1)
            fig.update_yaxes(title_text='Mean Height $\\langle h \\rangle$', row=2, col=1)

            fig.show()

widget = InterfaceGrowthWidget(title="KPZ Equation: Renormalization in SPDE")
widget.display()


## 8. The Optimal Basis: Karhunen–Loève Expansion & Gaussian Process Decomposition

Gaussian processes (including Brownian motion and its fractional variants) are infinitely dimensional objects. Yet, remarkably, their entire path structure can be captured by a countable orthogonal basis—the **Karhunen–Loève (KL) expansion**.

The KL theorem states that any second-order stochastic process $X(t)$ with covariance kernel $K(s,t)$ admits the representation:

$$X(t) = \mu(t) + \sum_{n=1}^\infty \sqrt{\lambda_n} \xi_n \phi_n(t)$$

where $\lambda_n, \phi_n$ are eigenvalues/eigenfunctions of the integral operator with kernel $K$, and $\xi_n \sim N(0,1)$ are independent.

This expansion is optimal: truncating at finite $N$ minimizes the mean-squared reconstruction error among all possible bases.

For Brownian motion and the Brownian bridge, the eigenfunctions are simple sines with explicitly known decay rates $\lambda_n \sim 1/n^2$. For fractional Brownian motion with Hurst parameter $H$, the decay is slower ($\sim 1/n^{2H+1}$), reflecting increased roughness.

The KL expansion thus provides the "principal components" of path space—analogous to how Borel resummation extracts the dominant physics from a divergent perturbative series, but here exactly and probabilistically.

This completes Level 5 beautifully as a "final boss" tying together stochastic paths, roughness, and optimal representation—perfect bridge to rough volatility applications where fractional Brownian drivers use KL for efficient simulation.



In [ ]:
# [Karhunen–Loève Path Decomposition. Simulation of Gaussian processes (Brownian motion, Brownian bridge, fractional Brownian motion). Users control the Hurst parameter H and truncation level N. The widget displays sample paths, the mean, the top eigenfunctions, and reconstructions from truncated KL modes, showing explained variance.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class KarhunenLoeveWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['process_type'] = ParameterSlider(
            name='process_type',
            value=0,
            min_val=0,
            max_val=2,
            step=1,
            description='Process (0: Brownian, 1: Bridge, 2: fBM)'
        )
        self.sliders['hurst'] = ParameterSlider(
            name='hurst',
            value=0.5,
            min_val=0.1,
            max_val=0.9,
            step=0.05,
            description='Hurst Parameter $H$'
        )
        self.sliders['trunc_level'] = ParameterSlider(
            name='trunc_level',
            value=10,
            min_val=1,
            max_val=50,
            step=1,
            description='Truncation Level $N$'
        )
        self.sliders['num_paths'] = ParameterSlider(
            name='num_paths',
            value=10,
            min_val=1,
            max_val=20,
            step=1,
            description='Sample Paths'
        )
        self.sliders['seed'] = ParameterSlider(
            name='seed',
            value=42,
            min_val=0,
            max_val=100,
            step=1,
            description='Random Seed'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            ptype = int(params['process_type'])
            H = float(params['hurst'])
            N_trunc = int(params['trunc_level'])
            num_paths = int(params['num_paths'])
            seed = int(params['seed'])

            np.random.seed(seed)

            # Time grid
            n_points = 200
            t = np.linspace(0, 1, n_points)
            dt = t[1] - t[0]

            # Compute covariance kernel
            T = t[:, np.newaxis]
            S = t[np.newaxis, :]

            if ptype == 0:  # Standard Brownian motion: K(s,t) = min(s,t)
                cov = np.minimum(T, S)
                title = 'Brownian Motion'
            elif ptype == 1:  # Brownian bridge: K(s,t) = min(s,t) - s*t
                cov = np.minimum(T, S) - T * S
                title = 'Brownian Bridge'
            else:  # Fractional Brownian motion
                # Corrected fBM covariance: K(s,t) = (1/2) * (|s|^(2H) + |t|^(2H) - |s-t|^(2H))
                # For domain [0,1], this simplifies
                cov = 0.5 * (np.abs(T)**(2*H) + np.abs(S)**(2*H) - np.abs(T - S)**(2*H))
                # Ensure numerical stability: clip to avoid negative values from rounding
                cov = np.maximum(cov, 0)
                title = f'Fractional Brownian Motion ($H={H:.2f}$)'

            # Eigen-decomposition of covariance matrix
            # Use symmetric eigh for better numerical stability
            eigvals, eigvecs = np.linalg.eigh(cov)

            # Sort in descending order
            idx = np.argsort(eigvals)[::-1]
            eigvals = eigvals[idx]
            eigvecs = eigvecs[:, idx]

            # Clip negative eigenvalues (numerical errors) to zero
            eigvals = np.clip(eigvals, 0, None)

            # Normalize eigenvectors (orthonormalize)
            norms = np.sqrt(np.sum(eigvecs**2, axis=0)) + 1e-12
            eigvecs = eigvecs / norms[np.newaxis, :]

            # Ensure proper scaling: eigenvectors should satisfy ∫ φ_i(t) φ_j(t) dt = δ_ij
            # With discrete grid, this means φ_i^T φ_j * dt = δ_ij
            # So we normalize by √(dt)
            eigvecs = eigvecs / np.sqrt(dt)

            # Generate sample paths using full KL expansion (for ground truth)
            # X(t) = Σ_n √(λ_n) ξ_n φ_n(t) where ξ_n ~ N(0,1)
            num_modes = len(eigvals)
            full_coeffs = np.random.randn(num_paths, num_modes) * np.sqrt(np.maximum(eigvals, 0))
            paths_full = full_coeffs @ eigvecs.T  # (num_paths, n_points)

            # Truncated reconstruction (using only first N_trunc modes)
            trunc_coeffs = full_coeffs[:, :N_trunc]
            paths_trunc = trunc_coeffs @ eigvecs[:, :N_trunc].T

            # Explained variance
            total_var = np.sum(eigvals)
            explained_var = np.sum(eigvals[:N_trunc]) / total_var if total_var > 1e-10 else 0.0

            # Mean path (should be zero for these processes)
            mean_full = np.mean(paths_full, axis=0)

            # Reconstruction error for each path
            errors = paths_full - paths_trunc
            mse_per_path = np.mean(errors**2, axis=1)
            mean_mse = np.mean(mse_per_path)

            # Create subplots: paths + eigenfunctions + error + variance
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=(
                    f'{title} Sample Paths ($N={N_trunc}$)',
                    'Top 5 Eigenfunctions $\\sqrt{\\lambda_n} \\phi_n(t)$',
                    'Reconstruction Error (Sample Path)',
                    'Eigenvalue Spectrum & Explained Variance'
                ),
                horizontal_spacing=0.12,
                vertical_spacing=0.15,
                specs=[[{"secondary_y": False}, {"secondary_y": False}],
                       [{"secondary_y": False}, {"secondary_y": False}]]
            )

            # Apply dark theme consistently
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=700,
                width=1100,
                showlegend=True
            )

            colors = self.plot_manager.config.colors

            # 1. Top-left: Sample paths (full thin, truncated thick)
            for i in range(num_paths):
                # Full paths (thin, faded)
                fig.add_trace(
                    go.Scatter(
                        x=t, y=paths_full[i],
                        mode='lines',
                        line=dict(color=colors[1], width=1),
                        opacity=0.3,
                        showlegend=False,
                        hoverinfo='skip'
                    ),
                    row=1, col=1
                )
                # Truncated paths (thick, solid)
                fig.add_trace(
                    go.Scatter(
                        x=t, y=paths_trunc[i],
                        mode='lines',
                        line=dict(color=colors[0], width=2),
                        name='Truncated ($N$ modes)' if i == 0 else None,
                        showlegend=(i == 0),
                        opacity=0.8 if i == 0 else 0.6
                    ),
                    row=1, col=1
                )

            # Mean path (white dashed)
            fig.add_trace(
                go.Scatter(
                    x=t, y=mean_full,
                    mode='lines',
                    line=dict(color='white', width=2.5, dash='dash'),
                    name='Mean',
                    showlegend=True
                ),
                row=1, col=1
            )

            # 2. Top-right: Top 5 eigenfunctions (scaled by √λ)
            top_k = min(5, N_trunc, len(eigvals))
            for k in range(top_k):
                if eigvals[k] > 1e-10:  # Only plot non-zero eigenvalues
                    scaled_eigfunc = np.sqrt(eigvals[k]) * eigvecs[:, k]
                    fig.add_trace(
                        go.Scatter(
                            x=t,
                            y=scaled_eigfunc,
                            mode='lines',
                            name=f'Mode ${k+1}$ ($\\lambda_{{{k+1}}}={eigvals[k]:.4f}$)',
                            line=dict(width=2),
                            showlegend=True
                        ),
                        row=1, col=2
                    )

            # 3. Bottom-left: Reconstruction error for first sample path
            sample_idx = 0
            error_path = errors[sample_idx]
            fig.add_trace(
                go.Scatter(
                    x=t, y=error_path,
                    mode='lines',
                    fill='tozeroy',
                    name='Error $e(t) = X(t) - X_N(t)$',
                    line=dict(color='red', width=2),
                    fillcolor='rgba(255,0,0,0.3)'
                ),
                row=2, col=1
            )
            # Add zero line
            fig.add_hline(
                y=0,
                line_dash="dot",
                line_color="gray",
                opacity=0.5,
                row=2, col=1
            )

            # 4. Bottom-right: Eigenvalue spectrum (bar + cumulative line)
            max_plot_modes = min(50, len(eigvals))
            mode_indices = np.arange(1, max_plot_modes + 1)

            # Bar chart of normalized eigenvalues
            normalized_evals = eigvals[:max_plot_modes] / total_var if total_var > 1e-10 else eigvals[:max_plot_modes]
            fig.add_trace(
                go.Bar(
                    x=mode_indices,
                    y=normalized_evals,
                    name='$\\lambda_n / \\sum \\lambda$',
                    marker_color=colors[2],
                    opacity=0.7
                ),
                row=2, col=2
            )

            # Cumulative variance line
            cum_var = np.cumsum(eigvals[:max_plot_modes]) / total_var if total_var > 1e-10 else np.cumsum(eigvals[:max_plot_modes])
            fig.add_trace(
                go.Scatter(
                    x=mode_indices,
                    y=cum_var,
                    mode='lines+markers',
                    name='Cumulative Variance',
                    line=dict(color='white', width=3),
                    marker=dict(size=4)
                ),
                row=2, col=2
            )

            # Vertical line at truncation level
            if N_trunc <= max_plot_modes:
                fig.add_vline(
                    x=N_trunc,
                    line_dash="dash",
                    line_color="yellow",
                    line_width=2,
                    annotation_text=f'$N={N_trunc}$',
                    annotation_position="top",
                    row=2, col=2
                )

            # Update axes labels
            fig.update_xaxes(title_text="Time $t$", row=1, col=1)
            fig.update_yaxes(title_text="Path Value $X(t)$", row=1, col=1)
            fig.update_xaxes(title_text="Time $t$", row=1, col=2)
            fig.update_yaxes(title_text="Scaled Eigenfunction $\\sqrt{\\lambda_n} \\phi_n(t)$", row=1, col=2)
            fig.update_xaxes(title_text="Time $t$", row=2, col=1)
            fig.update_yaxes(title_text="Error $e(t)$", row=2, col=1)
            fig.update_xaxes(title_text="Mode Index $n$", row=2, col=2)
            fig.update_yaxes(title_text="Normalized Variance", row=2, col=2)

            # Update layout with detailed title
            fig.update_layout(
                title=f'Karhunen–Loève Decomposition: {title}<br>'
                      f'Truncated at $N={N_trunc}$ modes ({explained_var*100:.1f}% variance explained, MSE = {mean_mse:.6f})',
                hovermode='x unified'
            )

            fig.show()

widget = KarhunenLoeveWidget(title="Karhunen–Loève Expansion of Gaussian Processes")
widget.display()


---

### References

* **Boltzmann, L.** (1872). *Weitere Studien über das Wärmegleichgewicht unter Gasmolekülen*.

* **Caffarelli, L., & Silvestre, L.** (2007). *An extension problem related to the fractional Laplacian*.

* **Hairer, M.** (2014). *A theory of regularity structures*.

* **Itô, K.** (1944). *Stochastic integral*.

* **Karhunen, K.** (1947). *Über lineare Methoden in der Wahrscheinlichkeitsrechnung*.

* **Kac, M.** (1949). *On distributions of certain Wiener functionals*.

* **Loève, M.** (1948). *Fonctions aléatoires du second ordre*.

* **Lyons, T.** (1998). *Differential equations driven by rough signals*.

* **Malliavin, P.** (1978). *Stochastic calculus of variation and hypoelliptic operators*.

* **Mandelbrot, B. B., & Van Ness, J. W.** (1968). *Fractional Brownian motions, fractional noises and applications*.

* **Vergara, R. C.** (2025). *Karhunen-Loève expansion of random measures*.
